In [22]:
import tkinter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import matplotlib.axes

## INTERFACE SETUP ##
root = tkinter.Tk()
root.title('Oxide Useer Interface') 
root.wm_title('Oxide Useer Interface')


def getPeak():
    max_Idif, max_T, max_index = -1, -1, -1
    bubble_threshold = 8
    for index, row in df.iterrows():
        if row['T'] > bubble_threshold:
            break
        if row['Idif'] > max_Idif:
            max_Idif, max_T, max_index = row['Idif'], row['T'], index
            
    return [max_Idif, max_T, max_index]

## LAYOUT ##

inputtxt = tkinter.Text(root, height = 2, width = 50) 
inputtxt.grid(row = 0, column = 0)


def _graph():
    plot = ax.plot(df_x, df_y)
    ax.set_ylabel('O2- concentration')
    ax.set_xlabel('Idif')
    ax.ticklabel_format(style='sci', scilimits=(0,0))
    #acquire peak
    peak_data = getPeak()
    
    calc_m = (df_y[1]-df_y[0])/(df_x[1]-df_y[0])
    calc_b = df_y[1] - calc_m*df_x[1]
    # peak_data[1]*0.001
    oxide_conc = calc_m*peak_data[0]+calc_b
    ax.scatter(peak_data[0], oxide_conc, c='red', marker='o')
    plot1 = df.plot(x='T', y='Idif', ax = ax1)
    plot1.set_ylabel('i')
    plot1.set_xlabel('V vs Pt')
    canvas.draw()
    canvas1.draw()
    
    #display values
    dispvals = 'Peak Potential: {} (V vs Pt), \n Peak Current: {} (A.cm-2), \n Oxide Concentration: {} (mol/kg)'
    allstr = dispvals.format(peak_data[2]*0.01, round(peak_data[0],5), round(oxide_conc*1000, 2))
    label = tkinter.Label(root, text= allstr)
    label.grid(row = 2, column = 0)
    
def _parse():
    input_val = inputtxt.get("1.0",'end-1c')
    #input_val = 'ModifiedSWV.xlsx'
    
    df = pd.read_excel(input_val, sheet_name='Sheet1')  
    df = df.drop([0])

    #plot data
    _graph()
    
button = tkinter.Button(master=root, text="Parse Data", command=_graph)
button.grid(row = 0, column = 1)

'''
def _quit():
    root.quit()     # stops mainloop
    root.destroy()
button = tkinter.Button(master=root, text="Quit", command=_quit)
button.pack()
'''


fig = Figure(figsize=(5,4), dpi=100)
ax = fig.add_subplot(111)

fig1 = Figure(figsize=(5,4), dpi=100)
ax1 = fig1.add_subplot(111)

## READ DATA ##

df = pd.read_excel('ModifiedSWV.xlsx', sheet_name='Sheet1')  
df = df.drop([0])

## PLOT DATA AND ADD TO INTERFACE ##

##calibration curve data ; modified based on curve data being used
#held constant --> can be changed via input
df_x = [0.00000000001, 0.0217]
df_y = [0.00000000001, 0.0403]

canvas = FigureCanvasTkAgg(fig, master=root)  # A tk.DrawingArea.
canvas.draw()
canvas.get_tk_widget().grid(row=1, column=0)

canvas1 = FigureCanvasTkAgg(fig1, master=root)  # A tk.DrawingArea.
canvas1.draw()
canvas1.get_tk_widget().grid(row=1, column=1)

button.configure(command=_parse)


root.mainloop() #root.mainloop() vs tkinter.mainloop()

In [15]:
df = pd.DataFrame()